In [1]:
import cv2 as cv
import pytesseract
from typing import OrderedDict
from math import ceil
import re

In [2]:
img = cv.imread('MicrosoftTeams-image (4).png')

In [3]:
image_data = pytesseract.image_to_data(img,  output_type=pytesseract.Output.DICT)
print(image_data['text'])


['', '', '', '', 'Tax', 'Invoice/Bill', 'of', 'Supply/Cash', 'Memo', '', 'amazon', '(Original', 'for', 'Recipient)', '', '', '', 'Diptly', 'Signed', 'by', '0S', 'Amazon', '', 'tre', '1003209', '05', 'PM', 'UTE', '', 'Favor', 'ice', '', 'Sold', 'By:', 'Billing', 'Address:', '', '‘Amazon', 'John', 'Doe', '', '36,', 'Green', 'Downtown,', 'Golden', 'Road,', 'FL', '36,', 'Green', 'Downtown,', 'Golden', 'Road,', 'FL', '', 'PAN', 'No:', 'XCV967729L', 'Shipping', 'Address:', '', 'John', 'Doe', '', '', '', 'GST', 'Registration', 'No:', '|X9879123TC', '', '36,', 'Green', 'Downtown,', 'Golden', 'Road,', 'FL', '', '', '', 'Invoice', 'Number:', 'FLR978298', '', 'Invoice', 'Details:', 'FL9898DKEFD', '', 'Invoice', 'Date:', '19.06.2019', '', '', '', '4', 'Healthsense', 'Forehead', 'Thermometer', 'with', 'fastreading', '—g45.69', '4', '§43.5', '9.00%', 'VAT', '$3.02', '$47', '', '', '', 'Order', 'Number:', 'A-8934792734', '', 'Order', 'Date:', '19.06.2019', '', '', '', '(white)', '889978312', '', 'Tot

In [4]:
data_dict = OrderedDict({
    'PAN':{'value':'','label': 'PAN','Pattern':'[A-Z]{3}\d{6}[A-Z]','zoning_type':'in_line'},
    'Billing':{'value':'','label': 'Billing','Pattern':'(.*)Shipping\s?Address','zoning_type':'left_bottom_large'},
    'Order_date':{'value':'','label': 'Date','Pattern':'\d{1,2}[\.-//]\d{1,2}[\.-//]\d{2,4}', 'zoning_type':'in_line'},
    'Gst':{'value':'','label': 'GST','Pattern':'\.?[A-Za-z]{1,2}\d{7}[A-Za-z]{2}','zoning_type':'in_line'},
    'Invoice Number':{'value':'','label': 'Invoice','Pattern':'[A-Za-z]{3}\d{6}','zoning_type':'in_line'},
    'Item':{'value':'','label': 'Description','Pattern':'(\w*)(Unit\s?Price|\$)','zoning_type':'left_bottom_right_large'},
    'Unit ':{'value':'','label': 'Unit', 'Pattern':'\$\d+\.?\d+', 'zoning_type':'bottom_large'},
    'Total':{'value':'','label':'Total', 'Pattern':'\$?\d{1,4}?\.?\d{1,4}?(\$\d+\.?\d+)','zoning_type':'in_line'}
})


In [5]:
for key in data_dict.keys():
    to_find = data_dict[key]['label'].lower()

    for i in range(len(image_data['text'])):
        if to_find in image_data['text'][i].lower():
            x,y,w,h = image_data['left'][i], image_data['top'][i], image_data['width'][i], image_data['height'][i]
            if data_dict[key]['zoning_type'] == "in_line":
                zoning_area = (x-10,y-10,w+x+430,h+y+10) 
            elif data_dict[key]['zoning_type'] == "left_bottom_right":
                zoning_area = (x-190,y-10,w+x+10,h+y+200) 
            elif data_dict[key]['zoning_type'] == "bottom_large":
                zoning_area = (x-w,y-10,w+x+w,h+y+50) 
            else:
                zoning_area = (x-150,y+20,w+x+430,h+y+100)
            temp_string = ""
            if data_dict[key]['value'] == "":
                for j in range(len(image_data['text'])):
                    x,y,w,h = image_data['left'][j], image_data['top'][j], image_data['width'][j], image_data['height'][j]
                    
                    if x>=zoning_area[0] and y>=zoning_area[1] and x+w <= zoning_area[2] and y+h <= zoning_area[3]:
                        temp_string += image_data['text'][j]+' '
                
                if re.findall(data_dict[key]['Pattern'], temp_string):

                    data_dict[key]['value'] = re.findall(data_dict[key]['Pattern'], temp_string)[0]
                    break
        

In [6]:
import json

In [7]:
f = open(f"{data_dict['Invoice Number']['value']}.json", 'w')
final_op = {key:data_dict[key]['value'] for key in data_dict.keys()}
# print("-",final_op)
json.dump(final_op, f)


f.close()
